# Trabalho Prático 2 - NLP
### Especificação:
Nosso objetivo é estudar a tarefa de POS Tagging para a língua Portuguesa. Para isso utilizaremos o corpus Mac-Morpho, produzido pelo grupo NILC da ICMC USP. O Mac-Morpho oferece dados para treinamento, validação e teste de modelos preditivos, capazes de classificar a classe gramatical de palavras em Português. Para acessar o conjunto de classes, acesse http://nilc.icmc.usp.br/macmorpho/macmorpho-manual.pdf.

O corpus está disponível em http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz. Você deverá implementar e avaliar a precisão de um modelo de POS tagging, a sua escolha. Voce pode utilizar pacotes que facilitem a implementação, como gensim, e transformers. Você deverá entregar documentação embutida em um notebook, detalhando a tarefa, a metodologia e os resultados. Sua análise deverá discutir quais as classes gramaticais para as quais a precisão é maior/menor. Não utilizaremos LLMs para essa tarefa, mas a sugestão é utilizar Transformers disponiveis e pre-treinados, em especial o BERT.


In [55]:
!pip install transformers pandas

In [56]:
import pandas as pd

### Leitura do Corpus

In [57]:
!wget http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
!tar zxvf macmorpho-v3.tgz

--2023-12-13 01:01:27--  http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
Resolving nilc.icmc.usp.br (nilc.icmc.usp.br)... 143.107.183.225
Connecting to nilc.icmc.usp.br (nilc.icmc.usp.br)|143.107.183.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2463485 (2.3M) [application/x-gzip]
Saving to: ‘macmorpho-v3.tgz.1’

macmorpho-v3.tgz.1  100%[===================>]   2.35M  1.05MB/s    in 2.2s    

2023-12-13 01:01:30 (1.05 MB/s) - ‘macmorpho-v3.tgz.1’ saved [2463485/2463485]

macmorpho-dev.txt
macmorpho-test.txt
macmorpho-train.txt


In [58]:
def read_files_to_df(tipo):
    filename = f"macmorpho-{tipo}.txt"
    words = []
    tags = []
    with open(filename, "r") as data:
        for line in data:
            linha = line.split()
            sentenca = ""
            tags_sentenca = []
            for word_tag in linha:
                word, tag = word_tag.split("_")
                sentenca += " " + word
                tags_sentenca.append(tag)
            words.append(sentenca)
            tags.append(tags_sentenca)

    return pd.DataFrame({'text': words, 'tag': tags})

# Ler o arquivo de teste e criar df
df_test = read_files_to_df("test")
df_test

,text,tag
0,Salto sete,"[N, ADJ]"
1,O grande assunto da semana em Nova York é a e...,"[ART, ADJ, N, PREP+ART, N, PREP, NPROP, NPROP,..."
2,"Número duplo especial , é inteirinho dedicado...","[N, ADJ, ADJ, PU, V, ADJ, PCP, PREP, N, PREP, ..."
3,"A endiabrada editora Tina Brown ex da "" Vanit...","[ART, PCP, N, NPROP, NPROP, N, PREP+ART, PU, N..."
4,Além das fotos de Richard Avedon .,"[PREP, PREP+ART, N, PREP, NPROP, NPROP, PU]"
...,...,...
9982,"2 Em abril do ano passado , o nacionalista ir...","[NUM, PREP, N, PREP+ART, N, PCP, PU, ART, N, A..."
9983,William Ross é deputado no Parlamento britâni...,"[NPROP, NPROP, V, N, PREP+ART, N, ADJ, PREP+AR..."
9984,"John Major , sem dúvida , vai querer uma prom...","[NPROP, NPROP, PU, PREP, N, PU, V, V, ART, N, ..."
9985,Mas se o tempo mostrar que o Ira realmente pa...,"[KC, KS, ART, N, V, KS, ART, NPROP, ADV, V, PR..."


### Importando modelo Pos-Tagger Portuguese

O modelo usado é um finetunning do modelo BERTimbau, treinado com o corpus MacMorpho.


In [59]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
import pandas as pd

# Carregar tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained("lisaterumi/postagger-portuguese")
model = AutoModelForTokenClassification.from_pretrained("lisaterumi/postagger-portuguese")


In [60]:
def pos_tagging(text):
    # Tokenizando o texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Inferência
    outputs = model(**inputs).logits

    # Obter o índice da maior logit (previsão)
    predictions = torch.argmax(outputs, dim=2)

    # Convertendo índice para a tag correspondente
    return [model.config.id2label[p.item()] for p in predictions[0]]


### Testando o modelo

In [61]:
df_test['predicted_tag'] = df_test['text'].apply(pos_tagging)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Tratando dataframe

In [62]:
def remove_tag_padding(l):
  return l[1:len(l)-1]


In [63]:
# criando coluna de tags previstas sem '<pad>'
df_test['clean_predicted'] = df_test['predicted_tag'].apply(remove_tag_padding)

In [64]:
# recebe coluna com labels verdadeiras e coluna com labels previstas,e retorna lista com todas labels previstas concatenadas
def concatena_labels(true_value, predicted):
  true_value_list = true_value.to_list()
  predicted_list = predicted.to_list()
  predicted_list_clean = []
  for i in range(len(true_value_list)):
    for j in range(len(true_value[i])):
      predicted_list_clean.append(predicted_list[i][j])
  return predicted_list_clean

predicted_list = concatena_labels(df_test['tag'], df_test['clean_predicted'])


In [65]:
# gerando lista com labels verdadeiras
ground_truth = []
l = df_test['tag'].to_list()
for j in range(len(l)):
  for i in range(len(l[j])):
    ground_truth.append(l[j][i])
ground_truth[:10]

['N', 'ADJ', 'ART', 'ADJ', 'N', 'PREP+ART', 'N', 'PREP', 'NPROP', 'NPROP']

In [66]:
from sklearn.metrics import accuracy_score
# Comparar com as tags verdadeiras e calcular a precisão
accuracy = accuracy_score(ground_truth, predicted_list)
print(f'Mean Accuracy: {accuracy}')

Mean Accuracy: 0.37988372679721705


In [67]:
from collections import defaultdict

def calculate_accuracy_by_class(true_labels, predicted_labels):
    # Inicializar dicionários para armazenar verdadeiros positivos, falsos positivos e falsos negativos
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)

    # Calcular verdadeiros positivos, falsos positivos e falsos negativos
    for true, pred in zip(true_labels, predicted_labels):
        if true == pred:
            true_positives[true] += 1
        else:
            false_positives[pred] += 1
            false_negatives[true] += 1

    # Calcular precisão por classe
    accuracies = {}
    for label in true_positives:
        # Precisão é verdadeiros positivos divididos pelo total de previsões positivas (verdadeiros positivos + falsos positivos)
        # para aquela classe
        total_predictions_for_class = true_positives[label] + false_positives[label]
        if total_predictions_for_class > 0:
            accuracies[label] = true_positives[label] / total_predictions_for_class
        else:
            accuracies[label] = 0.0  # Se não houve previsões para esta classe, definir precisão como 0

    return accuracies


# Calcular precisão por classe
accuracy_by_class = calculate_accuracy_by_class(ground_truth, predicted_list)

# Imprimir a precisão por classe
for label, accuracy in accuracy_by_class.items():
    print(f"Classe {label}: Precisão {accuracy:.2f}")

Classe N: Precisão 0.41
Classe ART: Precisão 0.46
Classe ADJ: Precisão 0.28
Classe PREP+ART: Precisão 0.34
Classe PREP: Precisão 0.36
Classe NPROP: Precisão 0.40
Classe V: Precisão 0.38
Classe PU: Precisão 0.41
Classe PCP: Precisão 0.24
Classe PROPESS: Precisão 0.45
Classe PROADJ: Precisão 0.40
Classe PDEN: Precisão 0.34
Classe KS: Precisão 0.36
Classe ADV: Precisão 0.41
Classe PROSUB: Precisão 0.51
Classe KC: Precisão 0.28
Classe PRO-KS: Precisão 0.26
Classe NUM: Precisão 0.21
Classe PREP+ADV: Precisão 0.58
Classe PREP+PROPESS: Precisão 0.35
Classe PREP+PROADJ: Precisão 0.48
Classe ADV-KS: Precisão 0.22
Classe PREP+PROSUB: Precisão 0.39
Classe IN: Precisão 0.27
Classe PREP+PRO-KS: Precisão 0.21
